In [16]:
### One cell to run everything

# # 1- Packages Import

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# # 2- Approach

# Here, we analyze the different targets for our predictive problem.

# The target is based on the following variable: "Hb_baseline_time".

# We don't go here into great depth about the interest of each target variable.

# # 3- Data import

# We import here our preprocessed masterfiles.

#for timelag in ['2','4','6','24']:
for timelag in ['2']:



    # In[193]:

    data = pd.read_csv('../data/master_file_different_hours/Preprocessed_Masterfile{}h.csv'.format(timelag), index_col=0)


   


    # # 4- Hemoglobin filter


    th_hb = 10




    data_filter = data.loc[data["Hb_value_initial"]<=th_hb]




    print("Shapes of non filter and filter for "+timelag+"hours:")
    print(data.shape)
    print(data_filter.shape)


    # # 5- Targets

    # ### 5.1- Split X-y
    
        
    def create_col_and_baseline(y,y_filter,funct,name):
        y['target_'+name]=y[new_col_target].apply(funct)
        y['target_'+name+'_baseline']=y['baseline_initial'].apply(funct)
        y_filter['target_'+name]=y_filter[new_col_target].apply(funct)
        y_filter['target_'+name+'_baseline']=y_filter['baseline_initial'].apply(funct)



    col_target = 'Hb_value_baseline'
    new_col_target = 'target_baseline'




    X = data.drop(col_target, axis=1)
    y = data[[col_target]]
    y = y.rename(columns={col_target: new_col_target})


    X_filter = data_filter.drop(col_target, axis=1)
    y_filter = data_filter[[col_target]]
    y_filter = y_filter.rename(columns={col_target: new_col_target})




    # ### 5.2- Targets preparation

    # Create "baseline" column

    y['baseline_initial']=data['Hb_value_initial']
    y_filter['baseline_initial']=data_filter['Hb_value_initial']
    
    
    # ##### Drop
    y['target_drop'] = y[new_col_target] - data['Hb_value_initial']
    y_filter['target_drop'] = y_filter[new_col_target] - data_filter['Hb_value_initial']

    
    # ##### Drop in %



    y['target_drop_percentage'] = y['target_drop']/y[new_col_target]
    y_filter['target_drop_percentage'] = y_filter['target_drop']/y_filter[new_col_target]


    # ##### Drop in % binary


    
    def is_low_percentage(chf):
        return int(abs(chf) <0.15)


    
    create_col_and_baseline(y,y_filter,is_low_percentage,'drop_percentage_bin')


    # ##### Drop in % category

    
    
    def is_low_percentage_cate(chf):
        if abs(chf) <0.10:
            return 0
        elif 0.10<=abs(chf) <0.20:
            return 1
        else:
            return 2

    create_col_and_baseline(y,y_filter,is_low_percentage_cate,'drop_percentage_cate')



    # ##### Binary hemoglobin


    def is_low(chf):
        return int(chf < 10)
    
    def is_low2(chf):
        return int(chf < 8)


        
    create_col_and_baseline(y,y_filter,is_low,'bin')
    create_col_and_baseline(y,y_filter,is_low2,'bin2')

    


    # ##### 3 categories hemoglobin


    def hemoglobin_cate(chf):
        if chf <9:
            return 0
        elif 9<= chf <12.5:
            return 1
        elif 12.5 <= chf:
            return 2

    
    create_col_and_baseline(y,y_filter,hemoglobin_cate,'cat')



    # ### 5.3- Targets overview


    y['target_baseline_baseline']=y['baseline_initial']
    y_filter['target_baseline_baseline']=y_filter['baseline_initial']
    
    cols_in_order= ['baseline_initial','target_baseline', 'target_baseline_baseline', 'target_bin',
       'target_bin_baseline', 'target_bin2', 'target_bin2_baseline',
       'target_cat', 'target_cat_baseline','target_drop',
       'target_drop_percentage', 'target_drop_percentage_bin',
       'target_drop_percentage_bin_baseline', 'target_drop_percentage_cate',
       'target_drop_percentage_cate_baseline']

    y=y[cols_in_order]
    y_filter=y_filter[cols_in_order]
    
    
    # # 6- Export


    export_name_X = '../data/X_not_imputed_{}h.csv'.format(timelag)
    X.to_csv(export_name_X)

    export_name_X_filter = '../data/X_filter_not_imputed_{}h.csv'.format(timelag)
    X_filter.to_csv(export_name_X_filter)



    export_name_y = '../data/y_targets_{}h.csv'.format(timelag)
    y.to_csv(export_name_y)

    export_name_y_filter = '../data/y_targets_filter_{}h.csv'.format(timelag)
    y_filter.to_csv(export_name_y_filter)



Shapes of non filter and filter for 2hours:
(6139, 59)
(1553, 59)


In [9]:
y.columns

Index(['target_baseline', 'baseline_initial', 'target_drop',
       'target_drop_percentage', 'target_drop_percentage_bin',
       'target_drop_percentage_bin_baseline', 'target_drop_percentage_cate',
       'target_drop_percentage_cate_baseline', 'target_bin',
       'target_bin_baseline', 'target_bin2', 'target_bin2_baseline',
       'target_cat', 'target_cat_baseline'],
      dtype='object')